## I. General Setup
1. [Download](http://downloads.esri.com/archydro/archydro/Setup/) ArcHydro tools
2. Remember each time you use ArcHydro to set target locations:
    1. Go to ApUtilities
    2. Select ArcHydro
    3. Set to directories where you want files stored
        * Make sure to create/use geodatabase for second location - otherwise you get error trying to do catchment polygon processing
3. Make sure everything is in GCS_WGS_1984

## II. Extracting MT Watersheds From MSU HUCS
1. Downloaded HUCS dataset
 - ftp://ftp.ftw.nrcs.usda.gov/wbd/WBD_Latest_Version_March2015/
2. Open in arcgis
3. Attirbutes -> select "States LIKE '%MT%'"
4. Editor -> Start Editing
5. Editor -> Merge
6. Right click layer -> Data -> Export Data

## III. Clipping Hydrosheds with HUCS Montana shapefile
1. Load Hydrosheds
2. Load HUCS file, reproject into same coordinate system as Hydrosheds
    * GCS_WGS_1984
3. Use data management -> raster -> raster Processing -> clip tool to clip hydrosheds to HUCS - 
 - check "Use Input Features for Clipping Geometry"

## IV. Generating the arcHydro Network
1. [Download](http://hydrosheds.cr.usgs.gov/dataavail.php) hydrosheds water direction and accumulation files
2. Clip files to HUCS shape using above clipping method
3. Set target locations as instructed above.
3. Generate stream definition using a threshold of 5. Can check to match with NHD network
4. Use stream segmentation, catchment grid delineation, catchment polygon proccessing, drainage line, and drainage point processing tools.
5. Use Hydro Network Generation tool to generate network

## Get huc code for each catchment
1. Join WBDHU10 huc shapefile and DranaigePoint based on location
2. Join Catchment and Drainage point, keep only HUC10 field

## Get Slope for each edge
1. Compute line parameters - will compute for drainage line
2. Join drainage line to catchment
3. Join Catchment to hydroedge

## V. Get stream order for hydroEdge layer
1. Set NextDownID's with NULL value to -1 by selecting the 'NextDownID IS NULL' and use field calculator to fill with -1
1. Use Assign River Order tool and create seperate StrahlerOrder field. '
2. Keep other field to use as order to compute
2. Join HydroEdge and Catchment layers

### Get gauge locations
1. Go to http://waterdata.usgs.gov/nwis/inventory
2. Select by state and site type
3. Select the states you want and chose "stream" in site type
4. Select view on map
5. When on map, click export and save as ESRI Shapefile

# VI. Convert decimal degrees to km by multiplying by 120, 14400 for squared measurements

## VI. Get Beaverhead Basin Shapefile
1. Load WBDHUC6 from NHD database, reproject, and then filter to Missouri Headwaters polygon
2. Clip Catchments, Polygons, DEM, and hillshade with bhPolygon

## Create Grid for Gridlist
1. Selelct catchments you want
2. Export catchments
3. Select all features
4. Editor -> merge
5. Clip dem with merged shapefile using data manange -> raster -> raster processing -> clip tool
6. Export as tif
7. Read into R and write coordinates to text file. createGridlist.ipynb

## VII. Calculate mean annual discharge and velocity 

## Merging LPJ Outputs to single file
1. Read in seed grid in R and use dimensions to modify SR_grid_merge.txt given by Kristen.
2. Use her script to merge files, WARNING - takes like 5 hours

# ToDo


# Scratch Notes

Maybe have catchments

Inputs Values: Rs - surface runoff
    Loop through drainages starting with 

### Edge Attributes
- Length
- Slope
- Catchment / CatchmentID
- River Order
- color

### Node Attributes
- color


Outputs:

time series of water flow for each month





To move forward, I'd like us to:
1. For each month (m), sum total runoff (Rs) for each catchment (k)
2. Assume that all the catchment runoff can enter the stream/edge (Sriv) [so no routing or lag approaches]
3. Assign each edge an average annual discharge (Qout) from Raymond's study based on Strahler stream order
4. Sum water moving across edges as Sriv=(Qin-Qout)+Rs, where Qout=min(Qout-raymond, Sriv)
5. We'll compare the hydrographs and to account for no routing/lags, develop a simple correction factor (c) to scale Rs downward.
6. Other notes:
    -Rs is in mm/month, to convert to m3 we divide by 1000 to get m/month and multiple by catchment area in units of m2
    -Qout from Raymond et al. can be convert to mm/month by dividing by 0.001 to get mm3 and dividing by catchment area, and scaling by time
    - We don't need to know Sriv length or width, Qout is includes this information!
